In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 데이터 준비
iris = load_iris() # 꽃 받침과 꽃 잎 사이즈를 가지고 꽃의 종류를 결정

X = iris.data
y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# 학습 데이터와 테스트 데이터로 분리 => train_test_split()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=123)


X_scaled


# 얼굴인식 => ??? => 사람의 얼굴을 수치화 => Open CV => 무인차 (Open CV)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train) # train=모의고사 # 학습을 시킬 때는 학습 데이터만 제공

model.predict(X_test) # 예측을 시킬 때는 테스트 데이터만 제공

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"정확도 : {accuracy * 100}")


모델 학습과 모델 성능
심플하게 모든 것은 ML flow에게 맡긴다. => mlflow.autolog()
autolog에서 추적하지 못하는 다른 파라미터,메트릭,메타데이터 등등의 값을 수동으로 기록

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print("",mlflow.get_tracking_uri())

In [ ]:
import mlflow.sklearn

mlflow.autolog()

with mlflow.start_run(nested=True):
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"정확도 : {accuracy * 100}")


In [ ]:
exp = mlflow.set_experiment(experiment_name='iris_classification_experiment')

print(f"Name: {exp.name}")
print(f"ID: {exp.experiment_id}")
print(f"Location: {exp.artifact_location}")
print(f"Tags: {exp.tags}")
print(f"Lifecycle: {exp.lifecycle_stage}")
print(f"Create Timestamp: {exp.creation_time}")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

models = {
    "LogisticRegression" :LogisticRegression(
        max_iter=200, #최대 반복횟수"
        C=1.0, # 규제 강도(C값이 적을 수록 규제가 강화됨)
        solver='lbfgs', #최적화 알고리즘
        random_state=123
    ),
    "RandomForest" : RandomForestClassifier(
        n_estimators=100, #트리의 갯수
        max_depth=None,
        random_state=123
    ),
    "SVC" : SVC(
        kernel='linear', # linear, sigmoid, poly, rbf
        random_state=123
    ),
}   


In [ ]:
# 위 모델들을 한번씩 불러와서 (반복문) => 최고의 모델을 찾아내고, 해당 파라미터를 기록합니다.

mlflow.autolog()

best_accuracy = 0
best_model = None
best_model_name = None

with mlflow.start_run(nested=True):
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_model_name = model_name

        print(f"Model Name: {model_name}, Accuracy: {accuracy}")

        mlflow.log_param('best_model', best_model_name) # 파라미터 로그
        mlflow.log_metric('best_accuracy', best_accuracy) # 메트릭 로그

    print(f"Best Model Name: {best_model_name}, Best Accuracy: {best_accuracy}")
    

In [ ]:
mlflow.autolog()
# 전체 모델에 대해서 기록을 하고 싶은데?

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name, nested=True):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        #모델을 mlflow에 저장
        model_path = f"model/{model_name}"
        mlflow.sklearn.log_model(model, model_path)

        mlflow.log_param(f'{model_name}_param', model.get_params())
        mlflow.log_metric(f'{model_name}_accuracy', accuracy)

        print(f"Model Name: {model_name}, Accuracy: {accuracy}")



In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

def promote_to_staging(): pass # stage
def promote_to_production(): pass # production
def archive_model(): pass # archive: 모델 폐기 단계 
def register_model(): pass # register: 모델 등록 단계

